In [1]:
import pandas as pd
from torch.utils.data import DataLoader
import torch

In [2]:
data = pd.read_csv('../cleaned/usdrub_new.csv')

In [3]:
df = data[['close', 'high', 'open', 'low', 'vol']]
tensor = torch.tensor(df.values)
if torch.cuda.is_available():
    tensor = tensor.to("cuda")

In [4]:
import math
training_length = math.floor(len(tensor) * 0.8)
training_data, test_data = tensor[:training_length], tensor[training_length:]

In [5]:
train_dataloader = DataLoader(training_data, batch_size=8)
test_dataloader = DataLoader(test_data, batch_size=8)

In [6]:
next(iter(train_dataloader))

tensor([[3.4710e+01, 3.4934e+01, 3.4893e+01, 3.4585e+01, 7.6690e+03],
        [3.4750e+01, 3.4830e+01, 3.4730e+01, 3.4530e+01, 1.0000e+01],
        [3.5710e+01, 3.5864e+01, 3.5775e+01, 3.5670e+01, 1.5640e+03],
        [3.9887e+01, 3.9975e+01, 3.9717e+01, 3.9655e+01, 3.7600e+02],
        [3.9972e+01, 4.0156e+01, 3.9896e+01, 3.9688e+01, 9.0100e+02],
        [4.3032e+01, 4.3137e+01, 4.1455e+01, 4.1407e+01, 9.5184e+04],
        [4.3023e+01, 4.3032e+01, 4.2997e+01, 4.2997e+01, 2.9000e+01],
        [4.3587e+01, 4.3617e+01, 4.3176e+01, 4.3147e+01, 1.6348e+05]],
       device='cuda:0', dtype=torch.float64)

In [7]:
from CurrencyDataset import CurrencyDataset
cur = CurrencyDataset(csv_file='../cleaned/usdrub_new.csv')

In [8]:
dataloader = DataLoader(cur, batch_size=8)

In [9]:
next(iter(dataloader))

tensor([[3.4893e+01, 3.4934e+01, 3.4710e+01, 7.6690e+03],
        [3.4730e+01, 3.4830e+01, 3.4750e+01, 1.0000e+01],
        [3.5775e+01, 3.5864e+01, 3.5710e+01, 1.5640e+03],
        [3.9717e+01, 3.9975e+01, 3.9887e+01, 3.7600e+02],
        [3.9896e+01, 4.0156e+01, 3.9972e+01, 9.0100e+02],
        [4.1455e+01, 4.3137e+01, 4.3032e+01, 9.5184e+04],
        [4.2997e+01, 4.3032e+01, 4.3023e+01, 2.9000e+01],
        [4.3176e+01, 4.3617e+01, 4.3587e+01, 1.6348e+05]], dtype=torch.float64)